<a href="https://colab.research.google.com/github/danchaud-vincent/tensorflow-deep-learning/blob/main/07_food_vision_milestone_project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 07 - Food vision milestone Project 1

In the previous chapter (**transfer learning part 3: scaling up**) we build Food Vision mini: a transfer learning model which beat the original results of the [Food 101 paper](https://data.vision.ee.ethz.ch/cvl/datasets_extra/food-101/) with only 10% of the data.

Now, what would happen if we used all the data ?

Well, that's we're going to find out in this notebook!

We're going to be building Food Vision Big, using all of the data from the Food101 dataset.

This time **we've got the goal of beating [DeepFood](https://www.researchgate.net/publication/304163308_DeepFood_Deep_Learning-Based_Food_Image_Recognition_for_Computer-Aided_Dietary_Assessment), a 2016 paper which used a Convolutional Neural Network trained for 2-3 days to achieve 77.4% top-1 accuracy.

> **Note :** **Top-1 accuracy** means "accuracy for the top softmax activation value output by the model" (because softmax outputs a value for every class, but top-1 means only the highest one is evaluated). **Top-5 accuracy** means "accuracy for the top 5 softmax activation values output by the model", in other words, did the true label appear in the top 5 activation values? Top-5 accuracy scores are usually noticeably higher than top-1.


<table>
    <thead>
        <tr>
            <th></th>
            <th>🍔👁 Food Vision Big™</th>
            <th>🍔👁 Food Vision mini</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>Dataset source</td>
            <td>TensorFlow Datasets</td>
            <td>Preprocessed download from Kaggle</td>	
        </tr>
        <tr>
            <td>Train data</td>
            <td>75,750 images</td>
            <td>7,575 images</td>	
        </tr>
        <tr>
            <td>Test data</td>
            <td>25,250 images</td>
            <td>25,250 images</td>	
        </tr>
        <tr>
            <td>Mixed precision</td>
            <td>Yes</td>
            <td>No</td>	
        </tr>
        <tr>
            <td>Data loading</td>
            <td>Perform tf.data API</td>
            <td>TensorFlow pre-built function</td>	
        </tr>
        <tr>
            <td>Target results</td>
            <td>77,4% top-1 accuracy</td>
            <td>50,76% top-1 accuracy</td>	
        </tr>
    </tbody>
</table>

Alongside attempting to beat the DeepFood paper, we're going to learn about two methods to significantly improve the speed of our model training:
1. Prefetching
2. Mixed precision training

## Check GPU

For this notebook, we're going to be doing something different.

We're going to be using mixed precision training.

Mixed precision training was introduced in Tensorflow 2.4.0.

What does **mixed precision training** do?

Mixed precision training uses a combination of single precision (float 32) and half-precision (float16) data types to speed up model training (up 3x on modern GPUs).

We'll talk about this more later on but in the meantime you can read the [Tensorflow documention on mixed precision](https://www.tensorflow.org/guide/mixed_precision) for more details.

For now, before we can move forward if we want to use mixed precision training we need to make sure the GPU powering our Google Colab instance is compatible.

For mixed precision training to work, you need to access to a GPU with a compute compatibility score of 7.0+.

Google Colab offers P100, K80 and T4 GPUs, however, **the P100 and K80 are not compatible with mixed precision training.**

Therefore before we proceed we need to make sure we have **access to a Tesla T4 GPU in ou google Colab instance.**

If you're not using Google Colab, you can find a list of various Nvidia GPU compute capabilities on Nvidia's developer website.

In [3]:
# If using Google Colab, this should output "Tesla T4" otherwise, 
# you won't be able to use mixed precision training
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-277ec80a-f5cf-b3cf-ce2c-3e2a412c6e5b)


In [5]:
# Check TensorFlow version (should be 2.4.0+)
import tensorflow as tf
print(tf.__version__)

2.9.2


## Get Helper functions

We've created a series of helper function throughout the previous notebooks in the course. Instead of rewriting them (tedious), we'll import the `helper_functions.py` file from the Github repo.

In [7]:
# Get helper functions file
import os

if not os.path.exists("helper_functions.py"):
  !wget https://raw.githubusercontent.com/danchaud-vincent/tensorflow-deep-learning/main/utils/helper_functions.py
else:
  print("[INFO] 'helper_functions.py' already exists, skipping download.")

--2022-11-19 15:30:20--  https://raw.githubusercontent.com/danchaud-vincent/tensorflow-deep-learning/main/utils/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4405 (4.3K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]   4.30K  --.-KB/s    in 0s      

2022-11-19 15:30:20 (68.2 MB/s) - ‘helper_functions.py’ saved [4405/4405]



In [8]:
# import series of helper functions for the notebook
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys

## Use TensorFlow Datasets to Download Data